In [2]:
import pandas as pd

### Load patent id granted in 2012 and application id in 2017.

In [3]:
grants2012id_df = pd.read_csv('../data/grant2012_all.csv')

In [4]:
grants2012id_df = grants2012id_df.drop("Unnamed: 0", axis=1)

In [5]:
grants2012id_df.head()

,parsed
0,8166569
1,8166570
2,8166571
3,8166572
4,8166573


In [6]:
app2017id_df = pd.read_csv('../data/application2017_all.csv')

In [7]:
app2017id_df = app2017id_df.drop("Unnamed: 0", axis=1)

In [8]:
app2017id_df.head()

,app_id
0,14988955
1,15299629
2,15239362
3,15462276
4,14998443


### Filter citation of office action which is for 2017 application (only once)


citations.csv and office_actions.csv is large. Just filter app_id to reduce size. Do it only once

Below cell need memory more than 15GB.

In [17]:
citations = pd.read_csv("../data/citations.csv")

In [18]:
citations.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
0,12000001,20060218340,20060218340,NaN,NaN,NaN,0,1,0
1,12000001,2343564,2343564,NaN,NaN,NaN,1,0,0
2,12000001,6622200,6622200,NaN,NaN,NaN,1,0,0
3,12000001,6182004,6182004,NaN,NaN,NaN,0,1,0
4,12000001,2323432,2323432,NaN,NaN,NaN,1,0,0


In [23]:
citations.shape

(58862278, 9)

In [20]:
citations_2017 = pd.merge(citations, app2017id_df, on="app_id")

In [22]:
citations_2017.shape

(278403, 9)

In [21]:
citations_2017.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
0,12022531,20050240098,20050240098,NaN,NaN,NaN,0,1,0
1,12022531,20050148995,20050148995,NaN,NaN,NaN,0,1,0
2,12022531,5873827,5873827,NaN,NaN,NaN,1,0,0
3,12022531,6544185,6544185,NaN,NaN,NaN,1,0,0
4,12022531,6725083,6725083,NaN,NaN,NaN,1,0,0


In [9]:
sum(app2017id_df["app_id"]==12022531)

1

In [24]:
citations_2017.to_pickle("../data/citations_2017.dat")

### Filer office_actions of app_id is in 2017 (onlly once)

In [25]:
office_actions = pd.read_csv("../data/office_actions.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
office_actions.shape

(4384532, 23)

In [27]:
office_actions.head()

,app_id,ifw_number,document_cd,mail_dt,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,...,rejection_103,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type
0,14150981,100867762,CTFR,2015-10-15,2632,375,219000,0,0,0,...,1,0,0,0,0,0,0,1,2,1
1,14198961,100867788,CTFR,2015-10-15,2699,345,173000,0,0,0,...,1,0,0,0,0,0,0,0,2,1
2,13796589,100867794,CTNF,2015-10-15,3776,606,159000,0,0,0,...,1,0,0,0,0,0,0,0,3,3
3,14673475,100867844,CTNF,2015-10-15,2627,345,175000,0,0,0,...,1,1,1,0,0,0,1,0,4,1
4,14669113,100867850,CTNF,2015-10-15,2666,382,118000,0,0,0,...,0,1,1,0,0,0,0,0,0,1


In [28]:
office_2017 = pd.merge(office_actions, app2017id_df, on="app_id")

In [29]:
office_2017.shape

(26342, 23)

In [30]:
office_2017.to_pickle("../data/office_2017.dat")

### Load citations and office actions which app_id in 2017 only

To reduce footprint, restart kernel here and reload it.

In [9]:
citations_2017 = pd.read_pickle("../data/citations_2017.dat")

In [10]:
office_2017 = pd.read_pickle("../data/office_2017.dat")

### Match office action and citation for 2017 application 2012 grants

In [11]:
citations_2012_2017 = pd.merge(citations_2017, grants2012id_df.astype("str"), on="parsed")

In [12]:
citations_2012_2017.shape, citations_2017.shape

((6745, 9), (278403, 9))

## Data setup assumption

Download application of 2017 and grants of 2012, unzip xml, and place under data/

From here: https://bulkdata.uspto.gov/#pats

### Grants 2012

Use "Patent Grant Full Text Data (No Images) (JAN 1976 - PRESENT)".

- Download all zip of https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/2012/
   - Use shell script to download zip (https://colab.research.google.com/drive/1gJO0VQ72Xfr_SY2dwWyiTvczIv0_gd4L)
       - save to download.sh and run
- place xml under data/grants2012/
   - ex.data/grants2012/ipg120103.xml data/grants2012/ipg120221.xml
   - Total about 24GB

### Application 2017

Use "Patent Application Full Text Data (No Images) (MAR 15, 2001 - PRESENT)"

- Download all zip of https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2017/
   - Use above colab shell script again
- place xml under data/application/2017
    - ex. data/application2017/ipa170105.xml data/appli cation2017/ipa170223.xml
    - Total about 35GB

In [13]:
INDEX_PATH="../data/index"

In [10]:
# currenty lindxr is often updated, so install herer instead of include in Dockerfile

!go get github.com/karino2/lindxr

In [14]:
import subprocess

In [15]:
# lindxr index -indexdest index/grants2012 -pattern "<doc-number>" -target "../data/grants2012/ipg12011*.xml"

def build_index(indexdest, pattern, targetpat):
    subprocess.call(["lindxr", "index", "-indexdest", indexdest, "-pattern", pattern, "-target", targetpat])

In [16]:
def subfile(fpath, start, end):
    return subprocess.check_output(["lindxr", "sub", "-start", str(start), "-end", str(end), "-input", fpath],universal_newlines=True).split("\n")

### Create poor man's index (do it only once)

In [19]:
%%time

build_index(INDEX_PATH+"/grants2012/docnum/", "<doc-number>", "../data/grants2012/ipg*.xml")

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 1min 2s


In [17]:
%%time

build_index(INDEX_PATH+"/grants2012/uspatgra/", "<us-patent-grant", "../data/grants2012/ipg*.xml")

CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 59.3 s


In [18]:
%%time

build_index(INDEX_PATH+"/grants2012/close_uspatgra/", "</us-patent-grant>", "../data/grants2012/ipg*.xml")

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 57.1 s


# Load index from files

In [17]:
import glob

In [18]:
indexDNGrants2012 = glob.glob("../data/index/grants2012/docnum/*.idx")

## First load one index from file and filter only real doc-number index

In [19]:
def collect_lines(fpath):
    with open(fpath, "r") as f:
        return [int(l.split(":", 1)[0]) for l in f]

In [20]:
def collect_line_match_tupple(indexfile):
    with open(indexfile, "r") as f:
        return [(int(lnumstr), match) for (lnumstr, match) in (l.rstrip("\n").split(":", 1) for l in f)]

In [60]:
indexDNGrants2012[0]

'../data/index/grants2012/docnum/ipg120501.xml.idx'

In [20]:
fpath = indexDNGrants2012[0]

In [22]:
open_lines = collect_lines(fpath.replace("docnum", "uspatgra"))
close_lines = collect_lines(fpath.replace("docnum", "close_uspatgra"))

In [23]:
len(open_lines), len(close_lines)

(5520, 5520)

In [25]:
tups = collect_line_match_tupple(fpath)

In [26]:
len(tups)

201316

In [27]:
tups[0]

(8, '<doc-number>D0658346</doc-number>')

Retrieve doc-number. Filter non-number value and convert to int here.

In [21]:
import re

In [22]:
doc_num_pat = re.compile(r'<doc-number>([0-9]+)</doc-number>')

In [23]:
def filer_doc_number_only_number(tups):
    tups_ids = []
    for lnum, docline in tups:
        mat = doc_num_pat.match(docline)
        if mat:
            tups_ids.append((lnum, int(mat.group(1))))
    return tups_ids


In [31]:
tups_ids = filer_doc_number_only_number(tups)

In [32]:
len(tups_ids), tups_ids[0]

(126788, (16, 29385429))

### Filter doc-number which start from 5 line from &lt;us-patent-grant...&gt;

In [24]:
DOC_NUMBER_OFFSET_FROM_OPEN=5

In [25]:
def search_closest_at(open_lines, line, candidate):
    """
    return maximum index which satisfy open_lines[idx] < line.
    Search is started from candidate.
    """
    for i in range(candidate, len(open_lines)):
        if open_lines[i] > line:
            return i-1
    return len(open_lines)-1


In [26]:
def list_matched_open_idx(open_lines, tups_ids):
    """
    return index of open_lines which tups_ids belong to.
    """
    matched_opens = []
    candidate = 0
    for lnum, id in tups_ids:
        res = search_closest_at(open_lines, lnum, candidate)
        matched_opens.append(res)
        candidate = res
    return matched_opens

In [36]:
matched_opens = list_matched_open_idx(open_lines, tups_ids)

### tupple of (line, doc-number, corresponding idx of open_lines)

In [27]:
def filter_offset_match_only(open_lines, tups_ids, matched_opens, expected_offset):
    return [(lnum, id, matched_opens[idx]) for idx, (lnum, id) in enumerate(tups_ids)
                if lnum == open_lines[matched_opens[idx]]+expected_offset ]

In [38]:
tups_only_5 = filter_offset_match_only(open_lines, tups_ids, matched_opens, DOC_NUMBER_OFFSET_FROM_OPEN)

In [39]:
len(tups_only_5), tups_only_5[0:5]

(4981,
 [(308158, 8166569, 539),
  (309038, 8166570, 540),
  (309428, 8166571, 541),
  (310710, 8166572, 542),
  (311438, 8166573, 543)])

### Check whether how many match to grant2012_all.csv

In [40]:
grants2012id = set(grants2012id_df["parsed"])

In [41]:
sum([id in grants2012id for _, id, _ in tups_only_5])

4981

In [42]:
# seems match to all, but filter for sure.
tups_only_5 = [(lnum, id, openidx) for lnum, id, openidx in tups_only_5 if id in grants2012id]

In [43]:
len(tups_only_5)

4981

### Filter docnumber only in citations_2012_2017

In [89]:
pat_id_set = set(citations_2012_2017["parsed"].astype("int"))

In [88]:
sum(citations_2012_2017["parsed"].astype("int").isna())

0

In [90]:
len(pat_id_set)

5424

In [91]:
final_tups = [(lnum, id, openidx) for lnum, id, openidx in tups_only_5 if id in pat_id_set]

In [92]:
len(final_tups)

98

In [94]:
final_tups[0:5]

[(359894, 8166649, 619),
 (426845, 8166734, 704),
 (445288, 8166761, 731),
 (447021, 8166763, 733),
 (603329, 8166969, 939)]

In [101]:
tups_only_5[0:5]

[(308158, 8166569, 539),
 (309038, 8166570, 540),
 (309428, 8166571, 541),
 (310710, 8166572, 542),
 (311438, 8166573, 543)]

In [102]:
sum(citations_2012_2017["parsed"].astype("int") == 8166649), sum(citations_2012_2017["parsed"].astype("int") == 8166569)

(1, 0)

### Make what I did so far as function

In [28]:
# doc_num_tupples is a list of (line, doc-number, corresponding idx of open_lines)
from collections import namedtuple

DocNumIndex = namedtuple('DocNumIndex', ['open_lines', 'close_lines', "docnum_tupples"])

In [29]:
def one_index_to_docid_tupples(fpath, patidset):
    """
    fpath: path for docnum index.
    patidset: Patent id set that we want to keep.
    """
    open_lines = collect_lines(fpath.replace("docnum", "uspatgra"))
    close_lines = collect_lines(fpath.replace("docnum", "close_uspatgra"))
    tups = collect_line_match_tupple(fpath)
    tups_ids = filer_doc_number_only_number(tups)
    matched_opens = list_matched_open_idx(open_lines, tups_ids)
    tups_only_5 = filter_offset_match_only(open_lines, tups_ids, matched_opens, DOC_NUMBER_OFFSET_FROM_OPEN)
    final_tups = [(lnum, id, openidx) for lnum, id, openidx in tups_only_5 if id in patidset]
    return DocNumIndex(open_lines, close_lines, final_tups)


In [105]:
docnum_ind = one_index_to_docid_tupples(fpath, pat_id_set)

In [108]:
len(final_tups), len(docnum_ind.docnum_tupples)

(98, 98)

In [109]:
docnum_ind.docnum_tupples[0:5]

[(359894, 8166649, 619),
 (426845, 8166734, 704),
 (445288, 8166761, 731),
 (447021, 8166763, 733),
 (603329, 8166969, 939)]

In [110]:
# confirm result by checking xml
_, patid, openidx = docnum_ind.docnum_tupples[0]

In [112]:
openidx

619

In [113]:
fpath

'../data/index/grants2012/docnum/ipg120501.xml.idx'

In [117]:
subfile("../data/grants2012/ipg120501.xml",
        docnum_ind.open_lines[openidx]-2, docnum_ind.close_lines[openidx])[0:10]

['<?xml version="1.0" encoding="UTF-8"?>',
 '<!DOCTYPE us-patent-grant SYSTEM "us-patent-grant-v42-2006-08-23.dtd" [ ]>',
 '<us-patent-grant lang="EN" dtd-version="v4.2 2006-08-23" file="US08166649-20120501.XML" status="PRODUCTION" id="us-patent-grant" country="US" date-produced="20120418" date-publ="20120501">',
 '<us-bibliographic-data-grant>',
 '<publication-reference>',
 '<document-id>',
 '<country>US</country>',
 '<doc-number>08166649</doc-number>',
 '<kind>B2</kind>',
 '<date>20120501</date>']

### Now do it for all files under grants2012

In [30]:
pat_id_set = set(citations_2012_2017["parsed"].astype("int"))

In [31]:
import tqdm

In [32]:
file_dni_dict = {}
for fpath in tqdm.tqdm(indexDNGrants2012):
    dni = one_index_to_docid_tupples(fpath, pat_id_set)
    if len(dni.docnum_tupples) == 0:
        print("skip {}".format(fpath))
    else:
        file_dni_dict[fpath] = dni

100%|██████████| 52/52 [00:27<00:00,  1.89it/s]


In [122]:
len(file_dni_dict.keys())

52

### Is patent id unique? (A. yes).

In [123]:
all_docnum = [patid for fpath in file_dni_dict.keys() for _, patid, _ in file_dni_dict[fpath].docnum_tupples]

In [124]:
len(all_docnum)

5424

In [125]:
len(set(all_docnum))

5424

### Then create dict of {patent-id: (path, openlinenum, closelinenum)}

In [33]:
def indexpath_to_xmlpath(fpath):
    return fpath.replace("data/index", "data").replace("docnum/", "")[:-4]

In [137]:
indexpath_to_xmlpath('../data/index/grants2012/docnum/ipg120821.xml.idx')

'../data/grants2012/ipg120821.xml'

In [35]:
patid_to_subfile = {patid: (indexpath_to_xmlpath(fpath), dni.open_lines[idx], dni.close_lines[idx])
                    for fpath, dni in file_dni_dict.items()
                    for _, patid, idx in dni.docnum_tupples}

In [140]:
len(patid_to_subfile.keys())

5424

In [141]:
list(patid_to_subfile.keys())[0:5]

[8249344, 8126465, 8232962, 8200771, 8118276]

In [142]:
patid_to_subfile[8249344]

('../data/grants2012/ipg120821.xml', 5342754, 5344633)

In [145]:
subfile(*patid_to_subfile[8249344])[0:10]

['<us-patent-grant lang="EN" dtd-version="v4.2 2006-08-23" file="US08249344-20120821.XML" status="PRODUCTION" id="us-patent-grant" country="US" date-produced="20120806" date-publ="20120821">',
 '<us-bibliographic-data-grant>',
 '<publication-reference>',
 '<document-id>',
 '<country>US</country>',
 '<doc-number>08249344</doc-number>',
 '<kind>B2</kind>',
 '<date>20120821</date>',
 '</document-id>',
 '</publication-reference>']

In [38]:
%%time
grants_all_xml_dict = {patid: "\n".join(subfile(fpath, openl-2, closel))
                       for patid, (fpath, openl, closel) in tqdm.tqdm(patid_to_subfile.items())}


  3%|▎         | 155/5424 [03:00<1:42:11,  1.16s/it]


  6%|▌         | 324/5424 [04:28<1:10:22,  1.21it/s]


  9%|▉         | 487/5424 [06:05<1:01:49,  1.33it/s]


 12%|█▏        | 661/5424 [07:01<50:36,  1.57it/s]


 15%|█▌        | 838/5424 [07:52<43:05,  1.77it/s]


...

 92%|█████████▏| 4972/5424 [23:22<02:07,  3.55it/s]


 95%|█████████▌| 5156/5424 [24:00<01:14,  3.58it/s]


 98%|█████████▊| 5339/5424 [24:38<00:23,  3.61it/s]


100%|██████████| 5424/5424 [24:56<00:00,  3.63it/s]


CPU times: user 26.4 s, sys: 1min 9s, total: 1min 35s
Wall time: 24min 56s


In [47]:
list(grants_all_xml_dict.keys())[0:5]

[8249344, 8126465, 8232962, 8148779, 8118276]

In [51]:
grants_all_df = pd.DataFrame({"parsed": list(grants_all_xml_dict.keys()), "xml": list(grants_all_xml_dict.values())})

In [52]:
grants_all_df.head()

,parsed,xml
0,8249344,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
1,8126465,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
2,8232962,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
3,8148779,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
4,8118276,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."


In [53]:
grants_all_df.to_pickle("../data/grants_2012_from2017_xmldf.dat")

check result.

In [54]:
one_xml = grants_all_df.iloc[1]["xml"].split("\n")

In [55]:
len(one_xml)

539

In [56]:
one_xml[0:10]

['<?xml version="1.0" encoding="UTF-8"?>',
 '<!DOCTYPE us-patent-grant SYSTEM "us-patent-grant-v42-2006-08-23.dtd" [ ]>',
 '<us-patent-grant lang="EN" dtd-version="v4.2 2006-08-23" file="US08126465-20120228.XML" status="PRODUCTION" id="us-patent-grant" country="US" date-produced="20120214" date-publ="20120228">',
 '<us-bibliographic-data-grant>',
 '<publication-reference>',
 '<document-id>',
 '<country>US</country>',
 '<doc-number>08126465</doc-number>',
 '<kind>B2</kind>',
 '<date>20120228</date>']

# Below here is obsolete

In [67]:
xmls = glob.glob("../data/grants2012/*.xml")

In [68]:
xmls[0]

'../data/grants2012/ipg120911.xml'

In [69]:
xmls[0][len("../data/"):]

'grants2012/ipg120911.xml'

In [70]:
xmlrels = [xml[len("../data/"):] for xml in xmls]

In [72]:
xmlrels[0]

'grants2012/ipg120911.xml'

In [73]:
%%time

for xmlrel in xmlrels:
    create_one_index(xmlrel)

CPU times: user 4.49 s, sys: 812 ms, total: 5.3 s
Wall time: 3min 48s


In [9]:
xmls = glob.glob("../data/application2017/*.xml")
xmlrels = [xml[len("../data/"):] for xml in xmls]

In [10]:
%%time

for xmlrel in xmlrels:
    create_one_index(xmlrel)

CPU times: user 9.32 s, sys: 1.43 s, total: 10.8 s
Wall time: 7min 2s


In [ ]:
    relxml = indexfile[len("../data/index/"):].rstrip(".idx")
    for tup in tupples:
        key = cut_head_zero_workaround(tup[1])
        dic.setdefault(key, []).append([relxml, int(tup[0])])

In [46]:
common = INDEX_PATH +'/grants2012/'

In [47]:
indexDNGrants2012[0][len(common):]

'docnum/ipg120501.xml.idx'

In [49]:
os.path.dirname(fpath)

'../data/index/grants2012/docnum'

In [50]:
os.path.basename(fpath)

'ipg120501.xml.idx'

In [51]:
indexUSPAGrants2012 = glob.glob("../data/index/grants2012/uspatgra/*.idx")

In [52]:
fpath = indexUSPAGrants2012[0]

In [59]:
with open(fpath, "r") as f:
    open_lines = [int(l.split(":", 1)[0]) for l in f]
with open(fpath.replace("uspatgra", "close_uspatgra"), "r") as f:
    close_lines = [int(l.split(":", 1)[0]) for l in f]
    

In [61]:
len(open_lines), len(close_lines)

(5520, 5520)

In [62]:
open_lines[0], close_lines[0]

(3, 364)

In [53]:
class SubFile:
    def __init__(self, fpath, start, end):
        self.start = start
        self.end = end
        self.parent = fpath
    def content():
        return subfile(fpath, start, end)

In [14]:
indexGrants2012 = {}

In [15]:
build_all_index_dict(indexfilesGrants2012, indexGrants2012)

In [7]:
# workaround. in *.xml, doc-number sometime starts with unknown 0, while citations.csv does not.
# so if digit is 8 and start from 0, cut first 0.

def cut_head_zero_workaround(key):
    if len(key) == 8 and key.startswith("0"):
        return key[1:]
    return key

In [8]:
# dict format:
# key is doc-number
# val is list of [relxml, linenumber]

def build_one_index_file_to_dict(indexfile, dic):
    with open(indexfile, "r") as f:
        tupples = [l.rstrip("\n").split(":", 1) for l in f]
    relxml = indexfile[len("../data/index/"):].rstrip(".idx")
    for tup in tupples:
        key = cut_head_zero_workaround(tup[1])
        dic.setdefault(key, []).append([relxml, int(tup[0])])

In [9]:
def build_all_index_dict(indexfiles, dic):
    for f in indexfiles:
        build_one_index_file_to_dict(f, dic)

In [10]:
indexfiles2017 = glob.glob("../data/index/application2017/*.idx")

In [11]:
index2017 = {}

In [12]:
build_all_index_dict(indexfiles2017, index2017)

In [13]:
indexfilesGrants2012 = glob.glob("../data/index/grants2012/*.idx")

In [14]:
indexGrants2012 = {}

In [15]:
build_all_index_dict(indexfilesGrants2012, indexGrants2012)

### Filter office action only of application 2017 doc-number.

In [16]:
# key of index2017 is string because doc-number sometime contains non-int value.
# so we create string row in office action data frame to use isin method.

office_15["app_id_str"] = office_15.app_id.map(str)

In [17]:
sum(office_15.app_id_str.isin(index2017) == True)

21820

In [18]:
office2017 = office_15[office_15.app_id_str.isin(index2017)]

In [19]:
len(office2017)

21820

In [20]:
office2017.head()

,app_id,ifw_number,document_cd,mail_dt,art_unit,uspc_class,uspc_subclass,header_missing,fp_missing,rejection_fp_mismatch,...,rejection_112,rejection_dp,objection,allowed_claims,cite102_gt1,cite103_gt3,cite103_eq1,cite103_max,signature_type,app_id_str
3561801,15005636,IKSG9XK1RXEAPX1,CTNF,2016-02-22,3766,600,509000,0,0,0,...,1,1,0,0,0,0,0,0,0,15005636
3564336,15001553,IKTT4P0TRXEAPX1,CTNF,2016-02-19,1625,514,279000,0,1,0,...,1,0,0,0,0,0,0,0,1,15001553
3568839,15009367,IKZGL7O1RXEAPX4,CTNF,2016-02-25,2137,711,103000,0,0,0,...,0,1,0,0,0,0,0,0,1,15009367
3569392,15041416,IKZI6SS3RXEAPX1,CTNF,2016-02-25,2618,345,619000,0,0,0,...,0,0,0,0,0,1,0,5,1,15041416
3574015,15014088,IL0U1CK0RXEAPX5,CTNF,2016-02-26,2852,399,085000,0,0,0,...,0,0,1,0,0,0,0,2,1,15014088


In [21]:
ifw_from_OA = set(office2017.ifw_number)

In [22]:
len(list(ifw_from_OA))

21820

In [23]:
citations_15.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
57732192,15000008,8291627,8291627,NaN,NaN,NaN,1,0,0
57732193,15000008,2093620,2093620,NaN,NaN,NaN,1,0,0
57732194,15000008,9221406,9221406,NaN,NaN,NaN,1,0,0
57732195,15000008,8544198,8544198,NaN,NaN,NaN,1,0,0
57732196,15000008,7225569,7225569,NaN,NaN,NaN,1,0,0


In [24]:
citations_2017 = citations_15[citations_15.ifw_number.isin(ifw_from_OA)]

In [25]:
len(citations_2017)

43416

In [26]:
citations_2017.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
57732253,15000027,8553488,8553488,IRPCOHEPRXEAPX4,102.0,a,1,0,1
57732273,15000034,Dando US 2015/0364400,20150364400,IYD99O6CRXEAPX0,103.0,NaN,0,0,1
57732274,15000034,Sun US 2012/0181696,20120181696,IYD99O6CRXEAPX0,103.0,NaN,0,0,1
57732275,15000034,Sun US 2012/0279760,20120279760,IYD99O6CRXEAPX0,103.0,NaN,0,0,1
57732324,15000048,20140091294,20140091294,IR59S77KRXEAPX5,103.0,NaN,1,0,1


In [27]:
# citations_we_have = citations_2017[citations_2017.citation_pat_pgpub_id.isin(indexGrants2012)]
citations_we_have = citations_2017[citations_2017.parsed.isin(indexGrants2012)]

In [28]:
len(citations_we_have)

8365

In [29]:
citations_we_have.head()

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
57733581,15000285,4659845,4659845,IV82WOB1RXEAPX5,103.0,NaN,1,0,1
57733622,15000297,7268118,7268118,IO1DLQ1SRXEAPX3,102.0,b,0,1,1
57733623,15000297,7268118,7268118,IO1DLQ1SRXEAPX3,103.0,a,0,1,1
57733624,15000297,5578570,5578570,IO1DLQ1SRXEAPX3,103.0,a,1,0,1
57733628,15000297,5342620,5342620,IO1DLQ1SRXEAPX3,103.0,a,1,1,1


In [30]:
citations_we_have.to_pickle("../data/citations_2017_2012.dat")

In [31]:
index2017_we_need = {str(key):index2017[str(key)] for key in set(citations_we_have.app_id)}

In [32]:
len(list(index2017_we_need.keys()))

4734

In [33]:
indexGrants2012_we_need = {str(key):indexGrants2012[str(key)] for key in set(citations_we_have.parsed)}

In [34]:
list(indexGrants2012_we_need.keys())[0:10]

['3087414',
 '5010940',
 '2916244',
 '20100012972',
 '20090300770',
 '6550847',
 '6597345',
 '5848419',
 '6415183',
 '6978184']

In [35]:
len(list(indexGrants2012_we_need.keys()))

6352

In [204]:
indexGrants2012_we_need['8227462']

[['grants2012/ipg121030.xml', 5731295],
 ['grants2012/ipg120724.xml', 3034084],
 ['grants2012/ipg120724.xml', 3038802]]

In [68]:
## cut subtext of (target-bound):(target+bound) from very large text file
## but if (target-bound) is smaller than 1, treat as 1.
## retuned value is tupple which value is
## first cell: offset for target
## second cell: array of text, each element stand for one line.

def retrieve_subtext_fast(target, fpath, begin_bound, end_bound):
    start = max(1, target-begin_bound)
    end = target+end_bound
    sed_arg = "'{},{}p'".format(start, end)
    subtext = !sed -n {sed_arg} {fpath}
    return (target - start, subtext)

In [98]:
#<us-patent-application ...>...</us-patent-application>
#<us-patent-grant ...>....</us-patent-grant>

def find_nearest_root_element(target_offset, subtext_arr, tagname):
    begin_tag = '<{}'.format(tagname)
    last_cand = -1
    for i, line in enumerate(subtext_arr):
        if i > target_offset:
            if last_cand == -1:
                # begin tag not found until target_offset.
                # We assume real doc-number must be near the begin_tag,
                # So we set BEGIN_BOUND small and this case is normal (doc-number is far away from begin tag, that is, wrong one.)
                return -1
            return last_cand
        if begin_tag in line:
            last_cand = i
    raise ValueError('no target tagname found around target_offset')        

In [101]:
def retrieve_begin_end(begin_offset, subtext_arr, tagname):
    end_tag = '</{}'.format(tagname)
    res = []
    for line in subtext_arr[begin_offset:]:
        res.append(line)
        if end_tag in line:
            return res
    raise ValueError('no end tagname found')        

In [39]:
import re

In [102]:
DOC_NUMBER_PAT=re.compile(r'<doc-number>([^<]+)</doc-number>')

def find_first_doc_number(text_arr):
    res = []
    for text in text_arr:
        matchObj = DOC_NUMBER_PAT.search(text)
        if matchObj:
            return cut_head_zero_workaround(matchObj.group(1))
    raise ValueError("No doc-number found.")
        
    

In [210]:
find_first_doc_number(res_pat)

'20170351915'

In [66]:
# tagname is "us-patent-application" or "us-patent-grant"
#<us-patent-application ...>...</us-patent-application>
#<us-patent-grant ...>....</us-patent-grant>

def get_patent(tagname, file_path, target_offset):
    BEGIN_BOUND=200
    END_BOUND=10000
    target_mod_offset, subtext_arr = retrieve_subtext_fast(target_offset, file_path, BEGIN_BOUND, END_BOUND)
    begin_pos = find_nearest_root_element(target_mod_offset, subtext_arr, tagname)
    if begin_pos == -1:
        return None, None
    result_patent = retrieve_begin_end(begin_pos, subtext_arr, tagname)
    docid = find_first_doc_number(result_patent)
    return docid, result_patent

In [42]:
patent_grants_dic = {}

In [64]:
def get_real_patent(tagname, key_cand, tup_list):
    for tup in tup_list:
        real_id, pat_text = get_patent(tagname, '../data/' + tup[0], tup[1])
        if real_id == key_cand:
            return pat_text
    return None

In [44]:
dict_list = list(indexGrants2012_we_need.items())

# Heuristics for optimization

If the same doc-number occure too much, it must be in patcite (not calim-ed patent).
We skip more than 5 occurence for a while.

But I check a few doc-number in grants2012, it always start with 08.
So print doc-number starts with 8 and more than 5 occurences and check manually.

In [73]:
import tqdm

In [87]:
patent_grants_dic = {}

In [88]:
for i, (key_cand, tup_list) in tqdm.tqdm(list(enumerate(dict_list))):
    # print("{}: {}, {}".format(i, key_cand, len(tup_list)))
    if len(tup_list) > 5:
        if key_cand.startswith("8"):
            print("Skip 8 beginning key cand: {}, {}".format(key_cand, len(tup_list)))
    else:
        real_pat = get_real_patent("us-patent-grant", key_cand, tup_list)
        if real_pat:
            patent_grants_dic[key_cand] = real_pat 
            print("found {}, {}, {}".format(i, key_cand, len(list(patent_grants_dic.keys()))))

  0%|          | 20/6352 [00:34<3:04:26,  1.75s/it]

found 19, 8202487, 1


  1%|          | 44/6352 [01:28<3:31:04,  2.01s/it]

found 43, 8114845, 2


  1%|▏         | 84/6352 [03:36<4:29:46,  2.58s/it]

...

found 1602, 8100537, 81


 25%|██▌       | 1608/6352 [2:40:01<7:52:08,  5.97s/it]

found 1607, 8139945, 82


 26%|██▌       | 1624/6352 [2:43:09<7:55:01,  6.03s/it]

Exception: no end tagname found

In [89]:
import pickle

In [90]:
len(patent_grants_dic)

82

In [91]:
with open("../data/patent_grants_82dic.dat", 'wb') as f:
    pickle.dump(patent_grants_dic, f)

In [93]:
with open("../data/patent_grants_candidate_tuplist.dat", 'wb') as f:
    pickle.dump(dict_list, f)

In [94]:
founded_keys = set(patent_grants_dic.keys())

In [97]:
[(key, len(tuplist)) for key, tuplist in dict_list if key in founded_keys]

[('8202487', 1),
 ('8114845', 2),
 ('8140012', 1),
 ('8144683', 1),
 ('8138863', 1),
 ('8095929', 2),
 ('8289665', 1),
 ('8087315', 2),
 ('8322805', 1),
 ('8147894', 1),
 ('8118868', 1),
 ('8101436', 1),
 ('8331747', 1),
 ('8239267', 1),
 ('8307045', 1),
 ('8169523', 1),
 ('8270788', 1),
 ('8199013', 1),
 ('8289837', 1),
 ('8123234', 1),
 ('8292411', 1),
 ('8166718', 1),
 ('8297738', 1),
 ('8106186', 1),
 ('8185850', 1),
 ('8201268', 2),
 ('8123510', 1),
 ('8206358', 1),
 ('8327128', 1),
 ('8263886', 1),
 ('8250085', 1),
 ('8140786', 3),
 ('8151694', 1),
 ('8291754', 1),
 ('8151323', 1),
 ('8113316', 1),
 ('8226943', 2),
 ('8106402', 1),
 ('8200899', 1),
 ('8163144', 1),
 ('8238313', 1),
 ('8325073', 1),
 ('8145369', 2),
 ('8129836', 1),
 ('8088025', 2),
 ('8132770', 1),
 ('8263863', 1),
 ('8329172', 1),
 ('8112505', 3),
 ('8274319', 1),
 ('8179078', 1),
 ('8173208', 1),
 ('8279279', 1),
 ('8234395', 2),
 ('8333054', 1),
 ('8108040', 1),
 ('8149102', 1),
 ('8150616', 1),
 ('8300293', 1

In [ ]:
for i, (key_cand, tup_list) in tqdm.tqdm(list(enumerate(dict_list))):
    if i <= 1607:
        continue
    if len(tup_list) > 5:
        if key_cand.startswith("8"):
            print("Skip 8 beginning key cand: {}, {}".format(key_cand, len(tup_list)))
    else:
        try:
            real_pat = get_real_patent("us-patent-grant", key_cand, tup_list)
            if real_pat:
                patent_grants_dic[key_cand] = real_pat 
                print("found {}, {}, {}".format(i, key_cand, len(patent_grants_dic)))
        except ValueError:
            print("No end tag found, skip. {}:{}".format(i, key_cand))
            


 25%|██▌       | 1610/6352 [00:01<00:03, 1371.87it/s]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



No end tag found, skip. 1625:2705223
No end tag found, skip. 1641:8198448
found 1647, 8126927, 83


 26%|██▌       | 1660/6352 [03:35<10:09,  7.69it/s]  

found 1659, 8098918, 84


 26%|██▌       | 1663/6352 [03:38<10:15,  7.62it/s]

...

 77%|███████▋  | 4898/6352 [7:55:37<2:21:11,  5.83s/it]

found 4897, 8246292, 281


 78%|███████▊  | 4928/6352 [8:00:57<2:18:58,  5.86s/it]

No end tag found, skip. 4927:6399146


 78%|███████▊  | 4943/6352 [8:02:56<2:17:39,  5.86s/it]

In [ ]:
3+4

In [112]:
len(patent_grants_dic)

351

In [113]:
with open("../data/patent_grants_dic_grants12_app17.dat", 'wb') as f:
    pickle.dump(patent_grants_dic, f)

In [114]:
list(patent_grants_dic.keys())[0:5]

['8277780', '8129444', '8299862', '8146514', '8148885']

In [116]:
# patent_grants_dic['8148885']

In [84]:
dict_list[83]

('8140012', [['grants2012/ipg120320.xml', 5011666]])

In [86]:
get_real_patent("us-patent-grant", dict_list[83][0], dict_list[83][1])

In [70]:
for i in range(9, 20):
    key_cand, tup_list = dict_list[i]
    print("{}: {}, {}".format(i, key_cand, len(tup_list)))

    real_pat = get_real_patent("us-patent-grant", key_cand, tup_list)
    if real_pat:
        print("found {}".format(i))

9: 6978184, 6
10: 20070233706, 1
11: 6063078, 40
12: 5980676, 1
13: 6365565, 1
14: 7080960, 1
15: 20090059936, 1
16: 20070203910, 1
17: 5907491, 31
18: 7268118, 2
19: 8202487, 1
found 19


In [71]:
for i in range(20, 30):
    key_cand, tup_list = dict_list[i]
    print("{}: {}, {}".format(i, key_cand, len(tup_list)))

    real_pat = get_real_patent("us-patent-grant", key_cand, tup_list)
    if real_pat:
        print("found {}".format(i))

20: 7797430, 4
21: 20100045635, 1
22: 7193644, 7
23: 6562906, 15
24: 5648506, 10
25: 6847481, 1
26: 5905777, 19
27: 5848173, 1
28: 4958625, 16
29: 5533521, 11


In [72]:
for i in range(30, 70):
    key_cand, tup_list = dict_list[i]
    print("{}: {}, {}".format(i, key_cand, len(tup_list)))

    real_pat = get_real_patent("us-patent-grant", key_cand, tup_list)
    if real_pat:
        print("found {}".format(i))

30: 7252600, 13
31: 7461077, 9
32: 6172344, 4
33: 5756981, 46
34: 6310839, 1
35: 6180415, 14
36: 20070231401, 1
37: 20080104348, 1
38: 20090138245, 1
39: 6064143, 1
40: 7583001, 1
41: 3675597, 3
42: 6054839, 1
43: 8114845, 2
found 43
44: D155065, 1
45: 20110193637, 1
46: 7579197, 1
47: 6222457, 3
48: 20110037114, 1
49: 4965097, 3
50: 5546932, 11
51: 5589033, 5
52: 20100270553, 1
53: 5281018, 4
54: 6458003, 1
55: 3804292, 2
56: 20050033234, 1
57: 6559731, 1
58: 8004092, 4
59: 5451176, 1
60: 7517363, 4
61: 6496766, 8
62: 20120240927, 1
63: 20090161602, 1
64: 5614206, 5
65: 5518597, 2
66: 3843288, 1
67: 4226236, 6
68: 4978144, 2
69: 7751806, 1


In [56]:
i

8

In [57]:
tup_list

[['grants2012/ipg121106.xml', 5342114],
 ['grants2012/ipg121002.xml', 5626822],
 ['grants2012/ipg120508.xml', 5794299],
 ['grants2012/ipg120508.xml', 5803487],
 ['grants2012/ipg120612.xml', 5407506],
 ['grants2012/ipg120522.xml', 5137393],
 ['grants2012/ipg121127.xml', 1444948],
 ['grants2012/ipg121127.xml', 6560493],
 ['grants2012/ipg120320.xml', 5202882],
 ['grants2012/ipg121016.xml', 6215625],
 ['grants2012/ipg120904.xml', 5781479],
 ['grants2012/ipg120814.xml', 5573793],
 ['grants2012/ipg120814.xml', 5576351],
 ['grants2012/ipg120828.xml', 596660],
 ['grants2012/ipg120828.xml', 5851701],
 ['grants2012/ipg120306.xml', 5691746],
 ['grants2012/ipg120306.xml', 5709177],
 ['grants2012/ipg120306.xml', 5720034],
 ['grants2012/ipg120327.xml', 5498384],
 ['grants2012/ipg120327.xml', 5508518],
 ['grants2012/ipg120327.xml', 5519359],
 ['grants2012/ipg120911.xml', 1390981],
 ['grants2012/ipg120911.xml', 5904173],
 ['grants2012/ipg120911.xml', 5944689],
 ['grants2012/ipg120605.xml', 5929238],
 

In [58]:
key_cand

'6415183'

In [69]:
real_pat = get_real_patent("us-patent-grant", key_cand, tup_list)
if real_pat:
    print("found {}".format(i))

### Below cell takes too much times. Must be bug.

In [ ]:
for key_cand, tup_list in indexGrants2012_we_need.items():
    try:
        real_pat = get_real_patent("us-patent-grant", key_cand, tup_list)
        if real_pat:
            patent_grants_dic[key_cand] = real_pat            
    except:
        print("can't find tag. ignore. {}".format(key_cand))

In [ ]:
len(list(patent_grants_dic.keys()))

In [205]:
real_id1, pat_text1 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg121030.xml', 5731295)
real_id1

['08300926', '12499630']

In [212]:
real_id2, pat_text2 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg120724.xml', 3034084)
real_id2

'8227462'

In [142]:
real_id3, pat_text3 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg121204.xml', 6903688)
real_id3

'08326698'

In [146]:
with open('tmp_out.txt', 'w') as f: 
    print("\n".join(pat_text1), file=f)

In [ ]:
['grants2012/ipg120417.xml', 6067926],
 ['grants2012/ipg121204.xml', 6903406],
 ['grants2012/ipg121204.xml', 6903688]

In [147]:
indexGrants2012_we_need['4242108']

[['grants2012/ipg121106.xml', 1672856],
 ['grants2012/ipg120703.xml', 1727841],
 ['grants2012/ipg120724.xml', 1752025]]

In [152]:
real_id1, pat_text1 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg121106.xml', 1672856)
real_id1

['08303685', '10565466']

In [153]:
real_id2, pat_text2 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg120703.xml', 1727841)
real_id2

['08211213', '12632244']

In [154]:
real_id3, pat_text3 = get_patent("us-patent-grant", '../data/' + 'grants2012/ipg120724.xml', 1752025)
real_id3

['08226748', '12602887']

In [170]:
indexGrants2012_we_need['20120181696']

KeyError: '20120181696'

In [171]:
indexGrants2012['20120181696']

KeyError: '20120181696'

In [155]:
list(indexGrants2012_we_need.keys())[5:20]

['6510334',
 '7534866',
 '8227462',
 '20110032802',
 '6768196',
 '5235967',
 '20110035525',
 '5708963',
 '8011686',
 '4254145',
 '7010112',
 '4116456',
 '4835565',
 '7500444',
 '20080181288']

In [167]:
'20150364400'.startswith("2015")

True

In [165]:
list(citations_2017.parsed)[0:5]

['8553488', '20150364400', '20120181696', '20120279760', '20140091294']

In [169]:
citations_2017[list(map(lambda s: s.startswith("2012"), [str(elm) for elm in list(citations_2017.parsed)]))]

,app_id,citation_pat_pgpub_id,parsed,ifw_number,action_type,action_subtype,form892,form1449,citation_in_oa
57732274,15000034,Sun US 2012/0181696,20120181696,IYD99O6CRXEAPX0,103.0,NaN,0,0,1
57732275,15000034,Sun US 2012/0279760,20120279760,IYD99O6CRXEAPX0,103.0,NaN,0,0,1
57732330,15000048,Ke U.S. Pub # Pub # 2012/0049221 A1,20120049221,IYEPR2ZURXEAPX4,103.0,NaN,0,0,1
57732335,15000048,Dai U.S. Pub # Pub # 2012/0087105 A1,20120087105,IYEPR2ZURXEAPX4,103.0,NaN,0,0,1
57733582,15000285,20120245377,20120245377,IV82WOB1RXEAPX5,103.0,NaN,1,0,1
57734766,15000519,20120261760,20120261760,J0B89O3HRXEAPX0,103.0,NaN,1,0,1
57737475,15000943,20120064136,20120064136,IOVFB1E5RXEAPX4,103.0,NaN,1,0,1
57739052,15001133,20120113095,20120113095,J3LJNGP5RXEAPX4,103.0,a,0,1,1
57739055,15001133,20120113095,20120113095,J3XHHLA6RXEAPX4,103.0,a,0,1,1
57739057,15001133,20120113095,20120113095,J3XHHLA6RXEAPX4,103.0,a,0,1,1


In [78]:
res_tup = retrieve_subtext_fast(6067926, 10, '../data/grants2012/ipg120417.xml')

In [79]:
res_tup[0]

10

In [80]:
res_tup[1][10]

'<doc-number>7945484</doc-number>'

In [83]:
list(index2017_we_need.keys())[0:5]

['15146848', '15192621', '15334971', '15049456', '15282523']

In [84]:
index2017_we_need['15146848']

[['application2017/ipa171207.xml', 5183637],
 ['application2017/ipa171207.xml', 5183673]]

In [91]:
res_tup = retrieve_subtext_fast(5183637, 5000, '../data/application2017/ipa171207.xml')

In [96]:
find_nearest_root_element(res_tup[0], res_tup[1], 'us-patent-application')

4667

In [93]:
res_tup[0]

5000

In [97]:
res_tup[1][4667]

'<us-patent-application lang="EN" dtd-version="v4.4 2014-04-03" file="US20170351915A1-20171207.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20171121" date-publ="20171207">'

In [103]:
res_pat = retrieve_begin_end(4667, res_tup[1], 'us-patent-application')

In [105]:
res_pat[0:5]

['<us-patent-application lang="EN" dtd-version="v4.4 2014-04-03" file="US20170351915A1-20171207.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20171121" date-publ="20171207">',
 '<us-bibliographic-data-application lang="EN" country="US">',
 '<publication-reference>',
 '<document-id>',
 '<country>US</country>']

In [104]:
res_pat[-1]

'</us-patent-application>'

In [ ]:
subtext = !sed -n ''

# Try and error

In [74]:
# create_one_index("grants2012/ipg120103.xml")

In [21]:
xmlrel = "application2017/ipa170105.xml"

In [22]:
xmlpath = "../data/"+xmlrel

In [29]:
indexfile=INDEX_PATH+xmlrel+".idx"

In [31]:
!mkdir -p {os.path.dirname(indexfile)}

In [32]:
!grep -n "doc-number" {xmlpath} | sed -e 's/\(.*\)<doc-number>\([^<]*\)<\/doc-number>.*/\1\2/' > {indexfile}

In [45]:
with open(indexfile, "r") as f:
    tupples = [l.rstrip("\n").split(":", 1) for l in f]

In [46]:
len(tupples)

31695

In [60]:
dic = {}

In [61]:
for tup in tupples:
    dic.setdefault(tup[1], []).append(int(tup[0]))

In [62]:
# some doc-number seems dup
dic["15265874"]

[3508, 3648]

In [ ]:
for tup in tupples:    
    if tup[1] not in dic:
        print(tup[1])
    dic[tup[1]] = 1    

### just read xml file is too slow. Give up using python

In [4]:
%%time

with open("../data/" + xmlpath, "r") as f:
    lnum = 0
    fpos = 0
    line = f.readline()
    while line:
        lnum += 1
        line = f.readline()
        fpos = f.tell()


KeyboardInterrupt: 

### Whoosh trial

Conclusion: indexing is too slow and we can't use it.

In [6]:
from whoosh.index import create_in
from whoosh.fields import *
import os, os.path


In [14]:
schema = Schema(path=ID(stored=True), line_num=NUMERIC(stored=True), pos=NUMERIC(stored=True), line=TEXT)

In [15]:
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

ix = index.create_in("indexdir", schema)

In [17]:
writer = ix.writer()

In [18]:
xmlpath = "application2017/ipa170105.xml"

In [21]:
with open("../data/" + xmlpath, "r") as f:
    lnum = 0
    fpos = 0
    line = f.readline()
    while line:
        lnum += 1
        writer.add_document(path=xmlpath, line=line, line_num=lnum, pos=fpos)
        line = f.readline()
        fpos = f.tell()

writer.commit()
ix.close()

KeyboardInterrupt: 